In [2]:
import featuretools as ft

In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.metrics import mean_squared_error


In [4]:
train = pd.read_csv("data/loan.csv", low_memory=False)


In [5]:
test = pd.read_csv("data/test.csv", low_memory=False)

In [6]:
metadata = pd.read_csv("data/description.csv",names="BrowseNotesFile")
metadata


/anaconda2/lib/python3.7/site-packages/pandas/io/parsers.py:678: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


,B,r,o,w,s,e,N,o.1,t,e.1,s.1,F,i,l,e.2
0,BrowseNotesFile,Description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,acceptD,The date which the borrower accepted the offer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,accNowDelinq,The number of accounts on which the borrower i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,accOpenPast24Mths,Number of trades opened in past 24 months.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,addrState,The state provided by the borrower in the loan...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,annual_inc_joint,The combined self-reported annual income provi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,annualInc,The self-reported annual income provided by th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,application_type,Indicates whether the loan is an individual ap...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,avg_cur_bal,Average current balance of all accounts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,bcOpenToBuy,Total open to buy on revolving bankcards.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 74 columns):
id                             887379 non-null int64
member_id                      887379 non-null int64
loan_amnt                      887379 non-null float64
funded_amnt                    887379 non-null float64
funded_amnt_inv                887379 non-null float64
term                           887379 non-null object
int_rate                       887379 non-null float64
installment                    887379 non-null float64
grade                          887379 non-null object
sub_grade                      887379 non-null object
emp_title                      835917 non-null object
emp_length                     842554 non-null object
home_ownership                 887379 non-null object
annual_inc                     887375 non-null float64
verification_status            887379 non-null object
issue_d                        887379 non-null object
loan_status          

In [8]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 73 columns):
id                             31 non-null int64
member_id                      31 non-null int64
loan_amnt                      31 non-null int64
funded_amnt                    31 non-null int64
funded_amnt_inv                31 non-null float64
term                           31 non-null object
installment                    31 non-null float64
grade                          31 non-null object
sub_grade                      31 non-null object
emp_title                      27 non-null object
emp_length                     31 non-null object
home_ownership                 31 non-null object
annual_inc                     31 non-null float64
verification_status            31 non-null object
issue_d                        31 non-null object
loan_status                    31 non-null object
pymnt_plan                     31 non-null object
url                            31 non-null objec

In [9]:
# drop id and member_id from training and test set, Drop X2, X3 as they are unique identifiers for each loan and borrower and not useful for our model
train.drop(['id', 'member_id'], axis=1, inplace=True)
test.drop(['id', 'member_id'], axis=1, inplace=True)

In [11]:
# remove '%' from interest rate column and change dtype to numeric
train['int_rate'] = pd.to_numeric(train['int_rate'])/100


In [12]:
train['int_rate'] = pd.to_numeric(train['int_rate'])/100


In [13]:

# drop the rows with missing interest rate values
train = train.dropna(subset=['int_rate'])

In [14]:
train.dropna(subset=['loan_amnt'], inplace=True)

In [15]:

# convert loan_amnt to numeric data type
test['loan_amnt'] = pd.to_numeric(test['loan_amnt'])

# fill rows in test data where there are missing values for loan_amnt column with median values
test['loan_amnt'] = test['loan_amnt'].fillna(test.loan_amnt.median(axis=0))

